In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as Axes3D
%matplotlib notebook

G =4.*np.pi**2

# Conversion factors
m_Sun = 1.98855e30 # Solar mass in kg
au_in_meters = 1.49598261e11 # 1 au in meters
year_in_seconds = 3600*24*365 # 1 year in seconds

In [2]:
def f(t, q0):
    '''
    q0: arreglo con las coordenadas-velocidades
    q0 = q0[N,6]
    
    q0[0] = [x0,y0,z0,vx0,vy0,vz0] : Sol
    q0[1] = [x0,y0,z0,vx0,vy0,vz0] : Tierra
    q0[2] = ....
    ...
    q0[N] = ...
    
    mass = [mSol, mTierra]
    
    dqdt = dqdt[N,6]
    '''
    N = len(mass) # Numero de particulas
    dqdt = np.zeros(q0.shape)
    
    # Ecuaciones Diferenciales
    
    dqdt[:,0:3] = q0[:,3:6] # Velocidades
    
    for i in range(N): # Aceleraciones
        Delta = q0[i,0:3] - q0[:,0:3]
        r = np.sqrt(np.sum(Delta*Delta, axis=1))
        r[i] = 1
        dqdt[i,3] = - G*np.sum(mass[:]*Delta[:,0]/r**3)
        dqdt[i,4] = - G*np.sum(mass[:]*Delta[:,1]/r**3)
        dqdt[i,5] = - G*np.sum(mass[:]*Delta[:,2]/r**3)
    
    return dqdt

In [3]:
def RK4(t0, q0, dt):
    k1 = f(t0         , q0)
    k2 = f(t0 + 0.5*dt, q0 + 0.5*k1*dt)
    k3 = f(t0 + 0.5*dt, q0 + 0.5*k2*dt)
    k4 = f(t0 + dt    , q0 + k3*dt)
    q1 = q0 + (dt/6)*(k1 + 2*k2 + 2*k3 + k4)
    return q1
    

In [4]:
(x,y,z,vx,vy,vz,mass) = np.loadtxt('sun_earth.asc', unpack = True)

x = x/au_in_meters
y = y/au_in_meters
z = z/au_in_meters

vx = vx*year_in_seconds/au_in_meters
vy = vy*year_in_seconds/au_in_meters
vz = vz*year_in_seconds/au_in_meters

mass = mass/m_Sun

N = len(mass)
# Malla temporal
t0 = 0.
tf = 3.
n = 1000

t_grid = np.linspace(t0, tf, n)
dt = t_grid[1] - t_grid[0]


FileNotFoundError: sun_earth.asc not found.

In [ ]:
q0 = np.asarray([x,y,z,vx,vy,vz])
q0 = q0.T

q = np.zeros([n, N, 6])

q[0,0] = q0[0]
q[0,1] = q0[1]

In [ ]:
# Loop Principal

for i in range(n-1):
    q[i+1] = RK4(t_grid[i], q[i], dt)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(q[:,0,0], q[:,0,1], q[:,0,2], color='crimson', marker='o', label='Sol')
ax.plot(q[:,1,0], q[:,1,1], q[:,1,2], color='cornflowerblue', label='Tierra')
ax.legend()
plt.show()